In [1]:
candidate_info = {
    "full_name": "Kelvin Prabhu",
    "email": "k.prabhu@example.com",
    "phone_number": "+1-555-782-1147",
    "years_of_experience": 3,
    "desired_positions": ["Backend Developer", "Machine Learning Engineer"],
    "current_location": "Bangalore, India",
    "tech_stack": {
        "languages": ["Python", "JavaScript"],
        "frameworks": ["Django", "FastAPI", "React"],
        "databases": ["PostgreSQL", "MongoDB"],
        "tools": ["Docker", "Git", "AWS"]
    }
}
def candidate_info_sentence(info):
    sentences = []
    sentences.append(f"{info['full_name']} has {info['years_of_experience']} years of experience in software development.")
    sentences.append(f"They are proficient in programming languages such as {', '.join(info['tech_stack']['languages'])}.")
    sentences.append(f"Their experience includes working with frameworks like {', '.join(info['tech_stack']['frameworks'])}.")
    sentences.append(f"They have hands-on experience with databases including {', '.join(info['tech_stack']['databases'])}.")
    sentences.append(f"Additionally, they are skilled in using tools such as {', '.join(info['tech_stack']['tools'])}.")
    sentences.append(f"Currently based in {info['current_location']}, they are seeking positions as a {', '.join(info['desired_positions'])}.")
    return "".join(sentences)
candidate_info_sentence(candidate_info)

'Kelvin Prabhu has 3 years of experience in software development.They are proficient in programming languages such as Python, JavaScript.Their experience includes working with frameworks like Django, FastAPI, React.They have hands-on experience with databases including PostgreSQL, MongoDB.Additionally, they are skilled in using tools such as Docker, Git, AWS.Currently based in Bangalore, India, they are seeking positions as a Backend Developer, Machine Learning Engineer.'

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.ai.generativelanguage_v1beta.types import Tool as GenAITool

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.2,
    max_retries=3,
    thinking_budget=1024,
)

E0000 00:00:1763528264.951122   19394 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=GROK_API_KEY,
    # other params...
)


In [49]:
system_prompt_text = """
You are TalentScout, an intelligent AI hiring assistant for a technology recruitment agency. 
Your role is to conduct the initial screening of candidates by having a structured two-step conversation:

STEP 1 — Information Gathering  
Collect the following required candidate details one by one:  
- Full Name  
- Email Address  
- Phone Number  
- Years of Experience  
- Desired Position(s)  
- Current Location  
- Tech Stack (programming languages, frameworks, databases, tools)  

Rules for STEP 1:  
- Ask for one field at a time.  
- If any field is missing, continue asking until all required information is complete.  
- Confirm correctness when unclear.  
- Store everything in memory so the conversation flows naturally.  

STEP 2 — Technical Question Generation  
Once the full tech stack is received, automatically generate 3–5 relevant technical questions 
based on the candidate’s declared technologies.  
Examples: Python, JavaScript, Django, Node.js, AWS, SQL, etc.  

Rules for STEP 2:  
- Questions must be practical and skills-focused.  
- Avoid generic definitions.  
- Tailor questions directly to the technologies the candidate listed.  

CONVERSATION BEHAVIOR  
- Stay on topic as a hiring assistant; do not role-play anything else.  
- Maintain context and continue the flow naturally.  
- Provide fallback guidance when user input is unclear.  
- End the conversation gracefully if the user says “bye”, “exit”, “quit”, or “stop”.  

PURPOSE  
Your goal is to collect complete candidate information and assess their technical depth through relevant questions.
"""


In [55]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_text),
    MessagesPlaceholder("history"),
    ("human", "{input}")
])


In [56]:
from langchain_core.chat_history import InMemoryChatMessageHistory

store = {}  # acts like long-term memory for sessions

def get_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


In [57]:
chain = prompt | llm

In [58]:
from langchain_core.runnables import RunnableWithMessageHistory

langmem_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key="input",
    history_messages_key="history"
)


In [ ]:


while True:
    user_input = input("You: ")
    if user_input.lower() in ["bye", "exit", "quit", "stop"]:
        print("TalentScout: Thank you for your time. Goodbye!")
        break
    response = langmem_chain.invoke(
        {"input": user_input},
        config={"configurable": {"session_id": "user1"}}
    )
    print(f"TalentScout: {response.content}")


Hello Kelvin Prabhu, nice to meet you. I'm TalentScout, an AI hiring assistant for a technology recruitment agency. I'll be guiding you through a two-step conversation to learn more about your background and assess your technical skills.

To get started, I'd like to collect some basic information from you. Can you please confirm that your full name is indeed Kelvin Prabhu?
TalentScout: Hello. I'm TalentScout, an AI hiring assistant for a technology recruitment agency. I'm here to conduct an initial screening to learn more about your background and assess your technical skills.

To get started, could you please tell me your full name?
TalentScout: We've already exchanged greetings. To move forward with the screening process, I need to collect some information from you. 

Could you please tell me your full name?
TalentScout: Thank you, A Anto Kelvin Prabhu. I've taken note of your full name.

Next, I'd like to know your email address. Could you please share it with me?
TalentScout: I've 

In [1]:
import pdfplumber
import re

def extract_clean_resume_text(pdf_path: str) -> str:
    """
    Extracts and cleans text from a PDF resume.
    
    Steps:
    - Extract raw text from all pages
    - Remove excessive spaces, newlines, tabs
    - Normalize bullet points and punctuation
    - Return a clean text string
    
    :param pdf_path: Path to the PDF file
    :return: Cleaned resume text as a string
    """
    raw_text = ""

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            txt = page.extract_text()
            if txt:
                raw_text += "\n" + txt

    # Cleaning
    cleaned = raw_text

    # Remove repeated whitespace
    cleaned = re.sub(r'\s+', ' ', cleaned)

    # Replace common bullet symbols with commas or separators
    cleaned = cleaned.replace("•", ", ").replace("●", ", ").replace("▪", ", ")

    # Fix spacing issues around punctuation
    cleaned = re.sub(r'\s+([.,!?])', r'\1', cleaned)
    
    # Trim leading/trailing spaces
    cleaned = cleaned.strip()

    return cleaned
Resume_text = extract_clean_resume_text("/mnt/c/Users/asus/Downloads/AANTOKELVINPRABHU (13).pdf")
print(Resume_text)

A Anto Kelvin Prabhu +91 8708687084 | antokelvin.prabhu@msam.christuniversity.in | linkedin.com/in/a-anto-kelvin-prabhu-48385b25a | github.com/kelvinprabhu | kelvinportfolio2071.netlify.app Professional Summary AI/ML Master’s Candidate with a Full-Stack Development background, specializing in designing end-to-end ML/DL solutions. Proven experience with Computer Vision, Retrieval-Augmented Generation (RAG) systems, and Intelligent Agents (LLMs, Memory, Planning, Workflow). Strong interest in Reinforcement Learning (RL) and Research applications. Education CHRIST (Deemed to be University) Bangalore, Karnataka Master of Science (Artificial Intelligence and Machine Learning) Jul 2024 – Present Kristu Jayanti University (Deemed to be University) Bangalore, Karnataka Bachelor of Computer Applications 2020 – 2023 Work Experience Software Developer (Full Stack Development) Jun 2023 – 2024 Camp Automation Bangalore,  Engineered and deployed scalable web and mobile solutions using React, React N

4513